In [2]:
import pandas as pd
import numpy as np
import os
import datetime
import matplotlib.pyplot as plt


import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

from skimage import color, exposure
from sklearn.metrics import accuracy_score

%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [3]:
cd '/content/drive/My Drive/Colab Notebooks/matrix/matrix_three/road_sign_prediction'

/content/drive/My Drive/Colab Notebooks/matrix/matrix_three/road_sign_prediction


In [0]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')

X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']


In [5]:
len(np.unique(y_train))

43

In [6]:
to_categorical(y_train)[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 1., 0.], dtype=float32)

In [0]:
if y_train.ndim == 1 : y_train = to_categorical(y_train)

In [8]:
print(y_train, y_train.shape)

[[0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]] (34799, 43)


In [0]:
if y_test.ndim == 1 : y_test = to_categorical(y_test)

In [0]:
input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [11]:
model = Sequential([
  Conv2D(filters=64, kernel_size=(3,3), activation='relu', input_shape=input_shape),

  Flatten(),

  Dense(num_classes, activation='softmax')
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 64)        1792      
_________________________________________________________________
flatten (Flatten)            (None, 57600)             0         
_________________________________________________________________
dense (Dense)                (None, 43)                2476843   
Total params: 2,478,635
Trainable params: 2,478,635
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='Adam')

In [13]:
#on colab to gpu to reduce time from 50s to 5s
model.fit(X_train,y_train)

1088/1088 [==============================] - 3s 3ms/step - loss: 10.7268 - accuracy: 0.7972


In [0]:
#function

def get_cnn_v1(input_shape, num_classes):
  return Sequential([
  Conv2D(filters=64, kernel_size=(3,3), activation='relu', input_shape=input_shape),

  Flatten(),

  Dense(num_classes, activation='softmax')
])


def train_model(model, X_train, y_train, params_fit={}):
  model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='Adam')

  #log directory 
  logdir = os.path.join('logs', datetime.datetime.now().strftime('%Y%m%d-%H%M%S'))
  #pass to log
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq= 1)

  model.fit(
      X_train,
      y_train,
      batch_size=params_fit.get('batch_size', 128),
      epochs = params_fit.get('epochs', 5),
      verbose = params_fit.get('verbose', 1),
      validation_data = params_fit.get('validation_data', (X_train, y_train)),
      callbacks = [tensorboard_callback]
      )
  
  return model

In [15]:
model = get_cnn_v1(input_shape, num_classes)
model_trained = train_model(model, X_train, y_train)

Epoch 1/5
272/272 [==============================] - 3s 10ms/step - loss: 20.4062 - accuracy: 0.7698 - val_loss: 0.2176 - val_accuracy: 0.9459
Epoch 2/5
272/272 [==============================] - 2s 9ms/step - loss: 0.1827 - accuracy: 0.9561 - val_loss: 0.2223 - val_accuracy: 0.9529
Epoch 3/5
272/272 [==============================] - 2s 9ms/step - loss: 0.1236 - accuracy: 0.9709 - val_loss: 0.1041 - val_accuracy: 0.9717
Epoch 4/5
272/272 [==============================] - 2s 9ms/step - loss: 0.1075 - accuracy: 0.9756 - val_loss: 0.1038 - val_accuracy: 0.9765
Epoch 5/5
272/272 [==============================] - 2s 9ms/step - loss: 0.1063 - accuracy: 0.9752 - val_loss: 0.0670 - val_accuracy: 0.9814


In [0]:
df=pd.read_csv('data/signnames.csv')
labels_dict = df.to_dict()['b']

In [0]:
def predict(model_trained, X_test, y_test, scoring = accuracy_score):

  y_test_norm = np.argmax(y_test, axis=1)

  y_pred_prob = model_trained.predict(X_test)
  y_pred = np.argmax(y_pred_prob, axis = 1)

  return scoring(y_test_norm, y_pred)


In [18]:
predict(model_trained, X_test, y_test)

0.8303854875283446

In [0]:
def train_and_predict(model):
  model_trained = train_model(model, X_train, y_train)
  return predict(model_trained, X_test, y_test)

In [20]:
def get_cnn_v3(input_shape, num_classes):
  return Sequential([
  Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
  MaxPool2D(),
  Dropout (0.3),

  Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
  MaxPool2D(),
  Dropout(0.3),

  Flatten(),

  Dense(1024,activation='relu'),
  Dropout(0.3),

  Dense(num_classes, activation='softmax')
])


train_and_predict( get_cnn_v3(input_shape, num_classes) )

Epoch 1/5
272/272 [==============================] - 3s 10ms/step - loss: 5.2564 - accuracy: 0.2084 - val_loss: 1.6782 - val_accuracy: 0.5603
Epoch 2/5
272/272 [==============================] - 2s 9ms/step - loss: 1.4712 - accuracy: 0.5771 - val_loss: 0.6746 - val_accuracy: 0.8245
Epoch 3/5
272/272 [==============================] - 2s 9ms/step - loss: 0.8876 - accuracy: 0.7352 - val_loss: 0.3771 - val_accuracy: 0.9136
Epoch 4/5
272/272 [==============================] - 2s 9ms/step - loss: 0.5736 - accuracy: 0.8276 - val_loss: 0.1823 - val_accuracy: 0.9665
Epoch 5/5
272/272 [==============================] - 2s 9ms/step - loss: 0.3987 - accuracy: 0.8814 - val_loss: 0.1150 - val_accuracy: 0.9762


0.8836734693877552

In [21]:
def get_cnn_v4(input_shape, num_classes):
  return Sequential([
  Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
  Conv2D(filters=32, kernel_size=(3,3), activation='relu'),
  MaxPool2D(),
  Dropout (0.3),

  Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
  Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
  MaxPool2D(),
  Dropout(0.3),

  Flatten(),

  Dense(1024,activation='relu'),
  Dropout(0.3),

  Dense(num_classes, activation='softmax')
])


train_and_predict( get_cnn_v4(input_shape, num_classes) )

Epoch 1/5
272/272 [==============================] - 3s 12ms/step - loss: 2.0490 - accuracy: 0.5262 - val_loss: 0.3329 - val_accuracy: 0.9229
Epoch 2/5
272/272 [==============================] - 3s 11ms/step - loss: 0.4397 - accuracy: 0.8720 - val_loss: 0.0905 - val_accuracy: 0.9815
Epoch 3/5
272/272 [==============================] - 3s 11ms/step - loss: 0.2481 - accuracy: 0.9258 - val_loss: 0.0574 - val_accuracy: 0.9862
Epoch 4/5
272/272 [==============================] - 3s 11ms/step - loss: 0.1701 - accuracy: 0.9496 - val_loss: 0.0306 - val_accuracy: 0.9939
Epoch 5/5
272/272 [==============================] - 3s 11ms/step - loss: 0.1284 - accuracy: 0.9614 - val_loss: 0.0201 - val_accuracy: 0.9951


0.9714285714285714

In [22]:
def get_cnn_v5(input_shape, num_classes):
  return Sequential([
  Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
  Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
  MaxPool2D(),
  Dropout (0.3),

  Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
  Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
  MaxPool2D(),
  Dropout(0.3),

  Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
  Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
  MaxPool2D(),
  Dropout(0.3),

  Flatten(),

  Dense(1024,activation='relu'),
  Dropout(0.3),

  Dense(1024,activation='relu'),
  Dropout(0.3),

  Dense(num_classes, activation='softmax')
])


train_and_predict( get_cnn_v5(input_shape, num_classes) )

Epoch 1/5
272/272 [==============================] - 4s 14ms/step - loss: 2.8003 - accuracy: 0.2360 - val_loss: 1.2946 - val_accuracy: 0.6065
Epoch 2/5
272/272 [==============================] - 4s 13ms/step - loss: 0.9949 - accuracy: 0.6806 - val_loss: 0.2734 - val_accuracy: 0.9179
Epoch 3/5
272/272 [==============================] - 4s 14ms/step - loss: 0.4488 - accuracy: 0.8585 - val_loss: 0.1315 - val_accuracy: 0.9614
Epoch 4/5
272/272 [==============================] - 4s 13ms/step - loss: 0.2821 - accuracy: 0.9127 - val_loss: 0.0541 - val_accuracy: 0.9844
Epoch 5/5
272/272 [==============================] - 4s 13ms/step - loss: 0.2043 - accuracy: 0.9376 - val_loss: 0.0394 - val_accuracy: 0.9889


0.9764172335600907

In [0]:
X_train_gray = color.rgb2gray(X_train).reshape(-1,32,32,1)
X_test_gray = color.rgb2gray(X_test).reshape(-1,32,32,1)

In [0]:
model = get_cnn_v5( (32,32,1), num_classes)
model_trained = train_model(model, X_train_gray, y_train, params_fit={})
predict(model_trained, X_test_gray, y_test)

Epoch 1/5
272/272 [==============================] - 4s 14ms/step - loss: 2.6638 - accuracy: 0.2534 - val_loss: 0.7604 - val_accuracy: 0.7476
Epoch 2/5
 43/272 [===>..........................] - ETA: 1s - loss: 0.8900 - accuracy: 0.7131

In [0]:
!git config --global user.email "kuband93@gmail.com"
!git config --global user.name "Jakub Andrzejewski"
!git add 4_data_augemntation.ipynb
!git commit -m 'reducing channels'
!git push -u origin master

[master 5319129] cnn before parameteres optimalization using gpu on google colab
 1 file changed, 1 insertion(+)
 create mode 100644 3_first_CNN_model.ipynb
